In [ ]:
### This code is for making 

In [6]:
### IMPORT MODULES ###
import os
os.chdir('../')
from module import libnc4
os.chdir('./driver')

### IMPORT LIBRARIES ###
import netCDF4 as nc
import matplotlib.pyplot as plt
import numpy as np

In [7]:
### DEFINE FILE PATHS ###
nc_path = '../data/ezaki_timeslice.nc'

### OPEN THE NETCDF FILE ###
dataset = nc.Dataset(nc_path, 'r')

### PRINT THE FILE AND VARIABLE INFORMATION ###
print("file information:")
print(dataset)
print("\nVariables:")
for variable in dataset.variables:
    print(variable)

file information:
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): z(500), nv(2), zh(501), x(384), xh(384), y(384), yh(384), CZ(504), FZ(505), FDZ(503), CX(388), CY(388), FX(389), FY(389), CXG(388), CYG(388), FXG(389), FYG(389), FDXG(389), FDYG(389), time(1)
    variables(dimensions): float32 T(time, z, y, x), float32 PRES(time, z, y, x)
    groups: 

Variables:
T
PRES


In [12]:
### GET A VARIABLE FROM THE NETCDF FILE ###
var_name = 'T'
# var_name = 'PRES'
data = dataset.variables[var_name][:]

# ### PRINT THE VARIABLE DATA ###
print(f"{var_name}:")
print(data)

### SHOW TEH SHAPE OF THE VARIABLE DATA ###
print(f"\nShape of the data: {data.shape}")

T:
[[[[292.1072  292.10944 292.1211  ... 292.1246  292.11923 292.11212]
   [292.11914 292.11136 292.10873 ... 292.11758 292.126   292.12567]
   [292.12515 292.12308 292.11758 ... 292.08975 292.10422 292.1189 ]
   ...
   [292.1418  292.13602 292.1388  ... 292.1411  292.1472  292.14774]
   [292.13727 292.1399  292.1443  ... 292.11868 292.1252  292.1324 ]
   [292.1175  292.1288  292.14246 ... 292.11218 292.10895 292.11044]]

  [[292.123   292.12936 292.14087 ... 292.13486 292.13184 292.1256 ]
   [292.13477 292.12488 292.12445 ... 292.1221  292.13507 292.13983]
   [292.13864 292.14056 292.1266  ... 292.09494 292.10675 292.1238 ]
   ...
   [292.21582 292.19772 292.15665 ... 292.18686 292.19824 292.20807]
   [292.1878  292.17676 292.14792 ... 292.15692 292.16528 292.17664]
   [292.1455  292.14893 292.14813 ... 292.13382 292.13126 292.1367 ]]

  [[292.15057 292.1418  292.14166 ... 292.15408 292.1595  292.15857]
   [292.14093 292.1371  292.1355  ... 292.11798 292.1373  292.14346]
   [292.1329 

In [29]:
# (time, z, y, x)
T = dataset.variables['T'][0,:,:,:]
T[90,:,:]

### GIVE NEW VARIABLE: HEIGHT ###
height_list = np.array([i for i in range(2, 1002, 2)])
height_list

array([   2,    4,    6,    8,   10,   12,   14,   16,   18,   20,   22,
         24,   26,   28,   30,   32,   34,   36,   38,   40,   42,   44,
         46,   48,   50,   52,   54,   56,   58,   60,   62,   64,   66,
         68,   70,   72,   74,   76,   78,   80,   82,   84,   86,   88,
         90,   92,   94,   96,   98,  100,  102,  104,  106,  108,  110,
        112,  114,  116,  118,  120,  122,  124,  126,  128,  130,  132,
        134,  136,  138,  140,  142,  144,  146,  148,  150,  152,  154,
        156,  158,  160,  162,  164,  166,  168,  170,  172,  174,  176,
        178,  180,  182,  184,  186,  188,  190,  192,  194,  196,  198,
        200,  202,  204,  206,  208,  210,  212,  214,  216,  218,  220,
        222,  224,  226,  228,  230,  232,  234,  236,  238,  240,  242,
        244,  246,  248,  250,  252,  254,  256,  258,  260,  262,  264,
        266,  268,  270,  272,  274,  276,  278,  280,  282,  284,  286,
        288,  290,  292,  294,  296,  298,  300,  3